# Example and exercise for using Pandas in the industry 

## Context 
There is a website where you could find a real-estate agent. There are a lot of agents listed on this website. So, this website decides to give some badges to the agents who perform well as a reward and meanwhile customers might find it easier to select a real-estate agent by referring to their badges. 

And the website decides to give out three kinds of badges:
* Most Trustworthy
* Most reviews 
* Highest Rate

Considering real-estate is a geological location sensitive business, this website decides to give out these badges at different geological levels. Besides, to be fair to the new agents, these badges are also given out at different time spans. 

Time spans:
* Last Year
* Last Half Year 
* Last Quarters
    * Q1, Q2, Q3 in current year
    * or Q4 in last year 
* Last Month

Geological scope:
* Country 
* City : e.g. Stockholm, Malmö, Nacka and etc ...
* Stockholm City
    * Geo type = 5: e.g. Norrmalm, Södermalm and etc ...
    * Geo type = 6: e.g. Vasastan, Södra-hammarbyhamnen, Gamla-stan and etc ...

<img style="float: middle;" src="images/real_estate.png" width="800">

## Input
There are six csv files include all the information this website could provide.

* 'agents.csv' includes all the detailed information for the real-estate agent on this website.

* ‘prepared_invite_for_agents.csv’ includes all the prepared-invites the website they send the agent. 

* ‘invites.csv’ includes all the invites these agents have sent to their clients. 

* ‘review.csv’ includes all the reviews these agents have received. 

* ‘location.csv’ includes Swedish location information this website is using.

* ‘zipcode,csv’ includes Swedish zip code information this website is using

## Output 
A csv file include all the badges required

## Suggested data process

* Part 1:
    * Read in relavent input csv files 
    * Check each input csv file and get familiar with the data
    * Get the logic for each type of badge ready
    
* Part 2:
    * Add more specific geographic information to the agent 
    * Get a list of time spans we will grant badges for

* Part 3:
    * Calculate the badge winner for each time span and geographic location.
    * Read out the result to a csv file

In [17]:
import pandas as pd
import numpy as np
import time
from datetime import datetime, timedelta
import glob
import warnings
warnings.filterwarnings("ignore")
###### import the fucntions in function.py #####
from functions import *

## Part 1:
### Trustworthy
* using prepared_invite_for_agents.csv 
* It includes all the prepared-invites the website they send the agent. These agents then could forward these prepared-invites to their clients to get a review. It does not matter whether this prepared invite will become a valid invite. 
    * id is unique for each prepared invite for agents. 
        * status: range from 1 to 5
        * status = 2 : this invite has been sent by the agent to the client 
        * status = 5 : this invite is invalid, has not reached to the agent
        * status = 1,3,4: this invite has reached to the agent, but not been sent to the client 

In [62]:
prepared_invite_for_agents = pd.read_csv("prepared_invite_for_agents.csv")
print ("There are %d prepared invites for agents"%len(prepared_invite_for_agents))
prepared_invite_for_agents.head()

There are 38429 prepared invites for agents


,id,created,modified,agent_id,status,agreement_date,street,person_type
0,3653,2017-05-05 06:46:08.417,2017-05-05 07:00:58.438,1652,1,2017-05-04 00:00:00,Dalslandsgatan 9,Seller
1,3659,2017-05-05 06:50:11.314,2017-05-05 07:00:58.45,1786,1,2017-05-04 00:00:00,Brännkyrkagatan 28,Buyer
2,3661,2017-05-05 06:50:11.327,2017-05-05 07:00:58.45,1786,1,2017-05-04 00:00:00,Brännkyrkagatan 28,Buyer
3,1228,2017-04-13 12:59:27.032,2017-05-05 07:09:08.48,2283,4,2016-04-27 00:00:00,Diligensvägen 132,Buyer
4,3672,2017-05-08 07:27:45.833,2017-05-08 07:31:19.554,2751,1,2017-05-07 11:28:57,Norr Mälarstrand 100,Buyer


#### Example of using the calculate_trust_worthy() in the function.py

In [16]:
calculate_trust_worthy(prepared_invite_for_agents).head()

,agent_id,number_total,number_sent,trust_worthy
365,4299,123,123.0,1.0
352,4207,100,100.0,1.0
245,3057,68,68.0,1.0
24,394,53,53.0,1.0
314,3892,51,51.0,1.0


#### Exercise
* Write your own calculate_trust_worthy() function
* Name it whatever you want 
* Generate the similar result as the example above

### Reviews
* using invites.csv
* It includes all the invites these agents have sent to their clients. Not all the invites will generate a valid review. 
    * Invite_id is unique for each invite
    * status: 
        * status = 2: this invite has no valid review 
        * status = 3: this invite has a valid review

In [10]:
invites = pd.read_csv("invites.csv")
invites = get_time_labels(invites)
print ("There are %d invites"%len(invites))
invites.head()

There are 6425 invites


,invite_id,created,modified,sentfrom,mobile,status,venue_name,venue_id,agent_id,agent_name,year,month,yymm
0,1737946,2018-03-05 14:55:20.571,2018-03-07 16:40:45.333,HusmanHagberg Farsta,NaN,3,NaN,4013504,NaN,NaN,2018,03,201803
1,1743142,2018-03-07 17:10:29.986,2018-03-09 17:10:39.676,Michaela Löfgren,NaN,2,Erik Olsson Fastighetsförmedling Göteborg,1831634,1640.0,Michaela Löfgren,2018,03,201803
2,1742840,2018-03-07 14:45:53.190,2018-03-07 17:22:16.455,Jenny Svensk,NaN,3,HusmanHagberg Värmdö,3549984,2823.0,Jenny Svensk,2018,03,201803
3,1743143,2018-03-07 17:10:30.054,2018-03-07 18:10:11.716,Michaela Löfgren,NaN,3,Erik Olsson Fastighetsförmedling Göteborg,1831634,1640.0,Michaela Löfgren,2018,03,201803
4,1741520,2018-03-07 10:14:01.825,2018-03-07 19:35:49.356,Johan Stenhardt,NaN,3,HusmanHagberg Vallentuna,1771962,5228.0,Johan Stenhardt,2018,03,201803


#### Example of using the calculate_no_reviews() in the function.py

In [18]:
calculate_no_reviews(invites).head()

,agent_id,number_reviews
0,29.0,7
1,56.0,3
2,108.0,5
3,240.0,13
4,242.0,3


#### Exercise
* Write your own calculate_no_reviews() function
* Name it whatever you want 
* Generate the similar result as the example above

### Rate
* using reviews.csv
* It includes all the reviews these agents have received.  
    * Invite_id is unique for each invite
    * status: 
        * status = 2: this invite has no valid review 
        * status = 3: this invite has a valid review

In [19]:
reviews = pd.read_csv("reviews.csv")
reviews = reviews.rename(columns={"agentid":"agent_id"})
print ("There are %d reviews"%len(reviews))
reviews.head()

There are 1728 reviews


,id,created,modified,rating,text,venueid,venuename,agent_id,agentname,invite_id
0,299635,2018-01-15 14:03:03.554,2018-01-15 14:03:21.895,5,Annas fenomenala insats gjorde vår bostadsaffä...,3669585,Våningen & Villan Lund,1221.0,Anna Ahlström,1641641
1,306311,2018-02-04 13:18:15.7,2018-02-04 13:18:16.827,5,"Mycket trevligt bemötande, kunnig och hjälpsam",3549970,HusmanHagberg Nacka,NaN,NaN,1665166
2,295624,2018-01-01 18:44:48.647,2018-01-01 18:45:13.2,5,Duktig mäklare! Patrik är både lyhörd och hjäl...,3616879,Våningen & Villan Malmö,2303.0,Patrik Truedsson,1566395
3,295641,2018-01-02 05:24:18.584,2018-01-02 05:24:48.022,5,"Mycket bra bemötande, man känner sig trygg och...",4013495,HusmanHagberg Karlstad,NaN,NaN,1573210
4,295760,2018-01-02 14:27:30.112,2018-01-02 14:28:20.353,1,Fick mycket gott första intryck av V&V. Mycket...,4020527,Våningen & Villan Malmö City,1220.0,Michaela Thulin,1590266


#### Example of using the calculate_avg_rating() in the function.py

In [21]:
calculate_avg_rating(reviews).head()

,agent_id,avg_rating,number_reviews
0,29.0,4.857143,7
1,56.0,3.250000,4
2,108.0,4.800000,5
3,240.0,4.846154,13
4,242.0,5.000000,4


#### Exercise
* Write your own calculate_avg_rating() function
* Name it whatever you want 
* Generate the similar result as the example above

## Part 2:
### Add more specific geographic information to the agent
* using agent.csv
    * has the postcode and city information of the venue where the agents work.
* location.csv
    * includes the name, id for each location at different geographical levels. 
    * location_id starts with its geo_type number .
    * Identifier is the specific name for each location, which is also  unique as the location id 

* zipcode.csv
    *  has the mapping between the zip code and locations at different geographical levels.
    
Location.csv and zipcode.csv is mainly used for finding the badge winner at Geo type 5 and 6 in Stockholm city.

In [27]:
### Read in agent info from agents.csv
agents = pd.read_csv("agents.csv")
print ("There are %d agents we are looking at"%len(agents))
agents.head()

There are 479 agents we are looking at


,agent_id,name,streetAddress,city,postalCode,lat,lng,venue_id,name.1,employeeState
0,7,Rikard Sölling,Edsbergs Torg 11,SOLLENTUNA,19252,59.445383,17.965861,3109176,Svenska Mäklarhuset Sollentuna,DELETED
1,8,Rickard Bornesand,Tuna Torg 1,VALLENTUNA,18631,59.533690,18.078277,3665641,Svenska Mäklarhuset Vallentuna,PUBLISHED
2,28,Jim Ribe,Sveavägen 52,Stockholm,11134,59.337814,18.061124,3040872,Edward & Partners,DELETED
3,29,Christian Melki,Sveavägen 52,Stockholm,11134,59.337814,18.061124,3040872,Edward & Partners,PUBLISHED
4,30,Edward Melki,Sveavägen 52,Stockholm,11134,59.337814,18.061124,3040872,Edward & Partners,PUBLISHED


#### data cleaning and pre-processing

In [111]:
### make sure all the cities are recorded in the same way
agents['city'] = agents['city'].str.lower()
agents['city'] = agents['city'].str.strip()

In [112]:
### select a subset of useful columns 
agent_info = agents[['city','agent_id','name','employeeState','venue_id','postalCode']]
agent_info.head()

,city,agent_id,name,employeeState,venue_id,postalCode
0,sollentuna,7,Rikard Sölling,DELETED,3109176,19252
1,vallentuna,8,Rickard Bornesand,PUBLISHED,3665641,18631
2,stockholm,28,Jim Ribe,DELETED,3040872,11134
3,stockholm,29,Christian Melki,PUBLISHED,3040872,11134
4,stockholm,30,Edward Melki,PUBLISHED,3040872,11134


#### Exercise
* Read in agents.csv 
* Generate a dataset as above 
* Add an extra column 'country' with value 'Sweden' to the dataset

In [113]:
location = pd.read_csv("location.csv")
location.head()

,id,location_id,geo_type,name,identifier,parents
0,869,300000,3,Stockholms län,stockholms-län,{}
1,870,300001,3,Uppsala,uppsala-län,{}
2,871,300002,3,Södermanland,södermanland,{}
3,872,300003,3,Östergötland,östergötland,{}
4,873,300004,3,Jönköping,jönköpings-län,{}


In [114]:
### Drop the row whose "county","municipality","locality" value is NaN
zipcode = pd.read_csv("zipcode.csv").dropna(subset=["county",
                                                    "municipality",
                                                    "locality"])
zipcode.head()

,id,zip_code,municipality,county,lat,lng,locality,locations
0,35408,11115,Lidingö,Stockholm,NaN,NaN,STOCKHOLM,"{300000,400018,440016,400021,500004,600021}"
1,35409,11120,Stockholm,Stockholm,59.3326,18.0649,STOCKHOLM,"{300000,440016,500001}"
2,35410,11121,Stockholm,Stockholm,59.3326,18.0649,STOCKHOLM,"{300000,440016,500001}"
3,35411,11122,Stockholm,Stockholm,59.3326,18.0649,STOCKHOLM,"{300000,440016,400020,500001,500013,600012}"
4,35412,11123,Stockholm,Stockholm,59.3326,18.0649,STOCKHOLM,"{300000,440016,500001,600012}"


#### Example of using the add_geo_info_for_agent() in the function.py
This self-defined function is mainly used to add extra geological information if the agents are working in Stockholm city since they will also compete for the badges at geo type = 5 and geo type = 6.

In [115]:
agent_info_geo = add_geo_info_for_agent(zipcode,location,agent_info)
agent_info_geo.head()

,city,agent_id,name,employeeState,venue_id,zip_code,location_id,geo_type,identifier,country
0,sollentuna,7,Rikard Sölling,DELETED,3109176,19252,NaN,NaN,NaN,Sweden
1,vallentuna,8,Rickard Bornesand,PUBLISHED,3665641,18631,NaN,NaN,NaN,Sweden
2,stockholm,28,Jim Ribe,DELETED,3040872,11134,500001,5.0,norrmalm,Sweden
3,stockholm,29,Christian Melki,PUBLISHED,3040872,11134,500001,5.0,norrmalm,Sweden
4,stockholm,30,Edward Melki,PUBLISHED,3040872,11134,500001,5.0,norrmalm,Sweden


#### Example of using the calculate_agent_geo_level() in the functions.py
To get a dict of agents we are intersted for different geo scopes

In [116]:
calculate_agent_geo_level(agent_info_geo)

{'country':      agent_id country employeeState
 0           7  Sweden       DELETED
 1           8  Sweden     PUBLISHED
 2          28  Sweden       DELETED
 3          29  Sweden     PUBLISHED
 4          30  Sweden     PUBLISHED
 ..        ...     ...           ...
 626      5387  Sweden     PUBLISHED
 627      5389  Sweden     PUBLISHED
 628      5390  Sweden     PUBLISHED
 629      5392  Sweden     PUBLISHED
 630      5393  Sweden     PUBLISHED
 
 [479 rows x 3 columns], 'city':      agent_id        city employeeState
 0           7  sollentuna       DELETED
 1           8  vallentuna     PUBLISHED
 2          28   stockholm       DELETED
 3          29   stockholm     PUBLISHED
 4          30   stockholm     PUBLISHED
 ..        ...         ...           ...
 626      5387       nacka     PUBLISHED
 627      5389      tyresö     PUBLISHED
 628      5390      tyresö     PUBLISHED
 629      5392      tyresö     PUBLISHED
 630      5393      tyresö     PUBLISHED
 
 [479 rows x 3 co

### Get a list of time spans we will grant badges for
* using prepared_invite_for_agents.csv for see what time spans we are interested in 

In [120]:
prepared_invite_for_agents.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38429 entries, 0 to 38428
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id              38429 non-null  int64         
 1   created         38429 non-null  datetime64[ns]
 2   modified        38429 non-null  object        
 3   agent_id        38429 non-null  int64         
 4   status          38429 non-null  int64         
 5   agreement_date  38424 non-null  object        
 6   street          38429 non-null  object        
 7   person_type     38429 non-null  object        
 8   year            38429 non-null  int64         
 9   month           38429 non-null  object        
 10  yymm            38429 non-null  object        
dtypes: datetime64[ns](1), int64(4), object(6)
memory usage: 3.2+ MB


#### Example of using the get_time_labels() in the functions.py
This self-defined function has very commonly used operations with datetime data type.

In [121]:
prepared_invite_for_agents_after = get_time_labels(prepared_invite_for_agents)
prepared_invite_for_agents_after.head()

,id,created,modified,agent_id,status,agreement_date,street,person_type,year,month,yymm
0,3653,2017-05-05 06:46:08.417,2017-05-05 07:00:58.438,1652,1,2017-05-04 00:00:00,Dalslandsgatan 9,Seller,2017,05,201705
1,3659,2017-05-05 06:50:11.314,2017-05-05 07:00:58.45,1786,1,2017-05-04 00:00:00,Brännkyrkagatan 28,Buyer,2017,05,201705
2,3661,2017-05-05 06:50:11.327,2017-05-05 07:00:58.45,1786,1,2017-05-04 00:00:00,Brännkyrkagatan 28,Buyer,2017,05,201705
3,1228,2017-04-13 12:59:27.032,2017-05-05 07:09:08.48,2283,4,2016-04-27 00:00:00,Diligensvägen 132,Buyer,2017,04,201704
4,3672,2017-05-08 07:27:45.833,2017-05-08 07:31:19.554,2751,1,2017-05-07 11:28:57,Norr Mälarstrand 100,Buyer,2017,05,201705


#### Exercise
* Write your own get_time_labels() function
* Name it whatever you want 
* Generate the similar result as the example above

#### Example of using the calculate_time_range_dic() in the functions.py
To get a list of time spans we are intersted for current data

In [122]:
time_now = prepared_invite_for_agents_after["created"].max()
time_now

Timestamp('2018-04-25 02:31:56.789000')

In [123]:
calculate_time_range_dic(prepared_invite_for_agents_after)

{'LastYear': ['201701',
  '201702',
  '201703',
  '201704',
  '201705',
  '201706',
  '201707',
  '201708',
  '201709',
  '201710',
  '201711',
  '201712'],
 'HalfYear': ['201707', '201708', '201709', '201710', '201711', '201712'],
 'Q1': ['201801', '201802', '201803'],
 'LastMonth': ['201803']}

#### Other solutions 
* Using pd.Period()
* Using conditions to get the badge winners for each time span. 

In [124]:
pd.Period(time_now,freq='Y')

Period('2018', 'A-DEC')

In [125]:
pd.Period(time_now,freq='Q').strftime('%YQ%q')

'2018Q2'

In [126]:
pd.Period(time_now,freq='M').strftime('%Y%m')

'201804'

## Part 3:
### Calculate the badge winner for each time span and geographic location.
#### Example of using fucntions in the function.py
##### 1. Add extra date info columns for the datasets

In [127]:
prepared_invite_for_agents = get_time_labels(prepared_invite_for_agents)
invites = get_time_labels(invites)
reviews = get_time_labels(reviews)
reviews = reviews.rename(columns={"agentid":"agent_id"})

##### 2. Generate the geo levels and time spans we are interested in

In [128]:
agent_geo_dic,time_range_dic = get_dic(agents,location,zipcode,prepared_invite_for_agents)

##### 3. Loop over the geo levels and time spans 

In [108]:
badges = pd.DataFrame()
for time_key in time_range_dic:
    time_range = time_range_dic[time_key]
    #print (time_range)
    for geo_key in agent_geo_dic:
        agent_geo = agent_geo_dic[geo_key]
        #print (geo_key)
        ################# Trust_worthy ##############
        tmp_result = get_trust_worth_list(prepared_invite_for_agents,time_range,time_key,agent_geo,geo_key)
        badges = badges.append(tmp_result)
        ################ Most reviews ###############
        tmp_result = get_most_review_list(invites,time_range,time_key,agent_geo,geo_key)
        badges = badges.append(tmp_result)
        ################ Highest rate ###############
        tmp_result = get_highest_rate_list(reviews,time_range,time_key,agent_geo,geo_key)
        badges = badges.append(tmp_result) 

##### 4. Save the result to a csv file

In [56]:
filename = 'example_result'
badges = badges.reset_index().rename(columns={"index":"geo_scope_name"})
badges["agent_id"] = badges["agent_id"].astype(int)
badges["badgeValue"] = badges["badgeValue"].round(4)
badges.to_csv(filename,index=False)
print ("Results have been save to "+ filename + '.csv')

Results have been save to example_result.csv


## Task

* Write your functions which could generate a similar result shown in the example
* You could skip the geo_scope for geo level 5 and level 6. Only calculate the badge winners at country and city level. 
* Compare your result with mine to see whether we got the same result. 

## End
Thanks for reading to the end and congratulations on your progress on using pandas to solve real world problems in the industry! 